## Find Best Locations in London to open Goila Butter Chicken's Cloud Kitchen and Compare with its Actual Location

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Extract London Neighborhoods and Get Location Coordinates</a>
    - Scrape Wikipedia page to get the London Underground Station Names (which will be our Neighborhoods)
    - Use Nominatim OpenStreet site to get coordinates for London and all its neighborhoods found above
    - Visualize the neighborhoods using Folium
    
      
2.  <a href="#item1">Explore and Cluster the Neighborhoods</a>
    - Use Foursquare API to get nearby venues and venue categories.  
    - Cluster the neighborhoods based on venue categories, using K-means algorithm 
    - Visualize the clustered neighborhoods using Folium
    - Get the top 10 most common venue categories in each neighborhood
    - Examine each cluster and determine how they are distinguished
  
  
3.  <a href="#item1">Recommend Locations to open Cloud Kitchen</a>  
    - Find top 5 neighborhoods with Indian Restaurants
    - Find the neighborhoods with Indian Restaurants amongst Top 10 venues
    - Observations and Conclusion

<a href="#item2"></a>

</font>
</div>

In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans

## Extract London Neighborhoods and Get Location Coordinates

####  Scrape Wikipedia page to get the London Underground Station Names (which will be our Neighborhoods)

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_London_railway_stations'

response = requests.get(url)
content = response.content
data = BeautifulSoup(content, 'html.parser')

In [4]:
tables = data.find_all('table')
len(tables)

13

In [5]:
london = pd.read_html(str(tables), flavor='bs4')[0]
london.head()

,Station,Local authority,Managed by,Stationcode,Farezone,Yearopened,Category,Coordinates
0,Abbey Wood,Greenwich,TfL Rail[1],ABW,4,1849,C,".mw-parser-output .geo-default,.mw-parser-outp..."
1,Acton Central,Ealing,London Overground,ACC,3,1853,D,51°30′32″N 0°15′48″W﻿ / ﻿51.5088°N 0.2634°W
2,Acton Main Line[2],Ealing,TfL Rail,AML,3,1868,E,51°31′01″N 0°16′01″W﻿ / ﻿51.5169°N 0.2669°W
3,Albany Park,Bexley,Southeastern,AYP,5,1935,D,51°26′09″N 0°07′36″E﻿ / ﻿51.4358°N 0.1266°E
4,Alexandra Palace[3],Haringey,Great Northern,AAP,3,1873,D,51°35′54″N 0°07′11″W﻿ / ﻿51.5983°N 0.1197°W


In [6]:
london.shape

(369, 8)

In [7]:
london['Station'] = london['Station'].str.split('[').str[0].str.strip()

In [8]:
london['Lat-Lon'] = london['Coordinates'].str.split('/').str[1].str.strip().astype(str)

In [9]:
london['Latitude'] = london['Lat-Lon'].str.split().str[0].str[:-2]

In [10]:
london['Longitude'] = london['Lat-Lon'].str.split().str[1].str[:-2]

In [11]:
london.head()

,Station,Local authority,Managed by,Stationcode,Farezone,Yearopened,Category,Coordinates,Lat-Lon,Latitude,Longitude
0,Abbey Wood,Greenwich,TfL Rail[1],ABW,4,1849,C,".mw-parser-output .geo-default,.mw-parser-outp...",﻿51.4915°N 0.1229°E,﻿51.4915,0.1229
1,Acton Central,Ealing,London Overground,ACC,3,1853,D,51°30′32″N 0°15′48″W﻿ / ﻿51.5088°N 0.2634°W,﻿51.5088°N 0.2634°W,﻿51.5088,0.2634
2,Acton Main Line,Ealing,TfL Rail,AML,3,1868,E,51°31′01″N 0°16′01″W﻿ / ﻿51.5169°N 0.2669°W,﻿51.5169°N 0.2669°W,﻿51.5169,0.2669
3,Albany Park,Bexley,Southeastern,AYP,5,1935,D,51°26′09″N 0°07′36″E﻿ / ﻿51.4358°N 0.1266°E,﻿51.4358°N 0.1266°E,﻿51.4358,0.1266
4,Alexandra Palace,Haringey,Great Northern,AAP,3,1873,D,51°35′54″N 0°07′11″W﻿ / ﻿51.5983°N 0.1197°W,﻿51.5983°N 0.1197°W,﻿51.5983,0.1197


In [12]:
london.groupby('Category')['Station'].count()

Category
??            1
A            12
A/Cd          1
B            12
C            85
C/Fe          1
D            85
E           122
F            29
unknown       1
unlisted     20
Name: Station, dtype: int64

In [13]:
london_neighborhoods = london[london['Category'].isin(['A','B'])][['Station','Latitude','Longitude']].copy()
london_neighborhoods.head()

,Station,Latitude,Longitude
9,Barking,﻿51.5393,0.0817
25,BlackfriarsLondon,﻿51.5116,0.1030
35,Bromley South,﻿51.4004,0.0181
46,Cannon StreetLondon,﻿51.5101,0.0912
58,Charing CrossLondon,﻿51.5075,0.1231


In [14]:
london_neighborhoods.reset_index(inplace=True)
london_neighborhoods.drop('index', inplace=True, axis=1)
london_neighborhoods.head()

,Station,Latitude,Longitude
0,Barking,﻿51.5393,0.0817
1,BlackfriarsLondon,﻿51.5116,0.1030
2,Bromley South,﻿51.4004,0.0181
3,Cannon StreetLondon,﻿51.5101,0.0912
4,Charing CrossLondon,﻿51.5075,0.1231


In [15]:
# Checking for any null values
london_neighborhoods.isnull().sum()

Station      0
Latitude     0
Longitude    0
dtype: int64

In [16]:
# Checking for any rows where Neighborhood is not assigned
london_neighborhoods[london_neighborhoods['Station'].isin(['Not Assigned','Not assigned','not assigned'])]

,Station,Latitude,Longitude


In [17]:
# Checking for unexpected values in Station Names
london_neighborhoods['Station'].unique()

array(['Barking', 'BlackfriarsLondon', 'Bromley South',
       'Cannon StreetLondon', 'Charing CrossLondon', 'Clapham Junction',
       'East Croydon', 'EustonLondon', 'Fenchurch StreetLondon',
       "King's CrossLondon", 'Liverpool StreetLondon',
       'London BridgeLondon', 'MaryleboneLondon', 'PaddingtonLondon',
       'Richmond', 'Shenfield', 'Stratford', 'Surbiton', 'VauxhallLondon',
       'VictoriaLondon', 'WaterlooLondon', 'Waterloo EastLondon',
       'Watford Junction', 'Wimbledon'], dtype=object)

In [18]:
london_neighborhoods['Neighborhood'] = london_neighborhoods['Station'].apply(lambda x : x.replace('London','') if x[0:6] != 'London' else x[:-6])

In [19]:
london_stations = london_neighborhoods['Neighborhood'].unique().tolist()
london_stations

['Barking',
 'Blackfriars',
 'Bromley South',
 'Cannon Street',
 'Charing Cross',
 'Clapham Junction',
 'East Croydon',
 'Euston',
 'Fenchurch Street',
 "King's Cross",
 'Liverpool Street',
 'London Bridge',
 'Marylebone',
 'Paddington',
 'Richmond',
 'Shenfield',
 'Stratford',
 'Surbiton',
 'Vauxhall',
 'Victoria',
 'Waterloo',
 'Waterloo East',
 'Watford Junction',
 'Wimbledon']

#### Use Nominatim OpenStreet site to get coordinates for London and all its neighborhoods found above

In [20]:
address = 'London, England'

url = 'https://nominatim.openstreetmap.org/search?q={}&format=jsonv2'.format(address)
response = requests.get(url)
content = response.json()

latitude = float(content[0]['lat'])
longitude = float(content[0]['lon'])
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [21]:
lat = []
lon = []
for station in london_stations:
    address = station + ', London, England'
    url = 'https://nominatim.openstreetmap.org/search?q={}&format=jsonv2'.format(address)
    response = requests.get(url)
    content = response.json()
    lat.append(float(content[0]['lat']))
    lon.append(float(content[0]['lon']))

london_neighborhoods['Latitude'] = lat
london_neighborhoods['Longitude'] = lon

In [22]:
london_neighborhoods.head()

,Station,Latitude,Longitude,Neighborhood
0,Barking,51.538992,0.080424,Barking
1,BlackfriarsLondon,51.511585,-0.103767,Blackfriars
2,Bromley South,51.400041,0.018277,Bromley South
3,Cannon StreetLondon,51.511381,-0.090363,Cannon Street
4,Charing CrossLondon,51.507497,-0.123689,Charing Cross


In [24]:
london_neighborhoods.shape

(24, 4)

#### Visualize the neighborhoods using Folium

In [25]:
# create map of New York using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area in zip(london_neighborhoods['Latitude'], london_neighborhoods['Longitude'], london_neighborhoods['Neighborhood']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Next, we will use the Foursquare API to explore the neighborhoods in above filtered boroughs and segment them.

## Explore and Cluster the Neighborhoods

#### Use Foursquare API to get nearby venues and venue categories

In [30]:
ID = '*****' # Foursquare ID
KEY = '*****' # Foursquare Key
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # Foursquare API limit value

In [31]:
cols = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

neighborhood_venues = pd.DataFrame(columns = cols)

In [32]:
for columns, neighborhood in london_neighborhoods.iterrows():
    radius = 2000
    neigborhood_name = neighborhood['Neighborhood']
    neighborhood_lat = neighborhood['Latitude']
    neighborhood_lon = neighborhood['Longitude']
        
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
    .format(ID, KEY, VERSION, neighborhood_lat, neighborhood_lon, radius, LIMIT)

    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    
    for v in venues:
        neighborhood_venues = neighborhood_venues.append({'Neighborhood': neigborhood_name, 
                                              'Neighborhood Latitude': neighborhood_lat,
                                              'Neighborhood Longitude': neighborhood_lon,
                                              'Venue': v['venue']['name'],
                                              'Venue Latitude': v['venue']['location']['lat'],
                                              'Venue Longitude': v['venue']['location']['lng'],
                                              'Venue Category': v['venue']['categories'][0]['name']}, ignore_index=True)

neighborhood_venues.head()             

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking,51.538992,0.080424,Barking Abbey,51.535352,0.076054,Park
1,Barking,51.538992,0.080424,Barking Park,51.545217,0.086134,Park
2,Barking,51.538992,0.080424,Cristina's,51.536523,0.076672,Steakhouse
3,Barking,51.538992,0.080424,The Gym London Barking,51.536193,0.078601,Gym
4,Barking,51.538992,0.080424,Nando's,51.539780,0.082297,Portuguese Restaurant


In [33]:
# Count of venues
neighborhood_venues.shape[0]

2316

In [34]:
# Count of unique venues
len(neighborhood_venues['Venue Category'].unique())

233

In [35]:
# Count of unique venue category in each neighborhood
neighborhood_venues.groupby('Neighborhood')['Venue Category'].count()

Neighborhood
Barking              54
Blackfriars         100
Bromley South        62
Cannon Street       100
Charing Cross       100
Clapham Junction    100
East Croydon        100
Euston              100
Fenchurch Street    100
King's Cross        100
Liverpool Street    100
London Bridge       100
Marylebone          100
Paddington          100
Richmond            100
Shenfield           100
Stratford           100
Surbiton            100
Vauxhall            100
Victoria            100
Waterloo            100
Waterloo East       100
Watford Junction    100
Wimbledon           100
Name: Venue Category, dtype: int64

#### Cluster the neighborhoods based on venue categories, using K-means algorithm

In [36]:
# one hot encoding
venues_encoded = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe at first position
venues_encoded.insert(0, 'Neighborhood', neighborhood_venues['Neighborhood'].values)

venues_encoded.head()

,Neighborhood,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus School,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Field,Hockey Rink,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soccer Stadium,Social Club,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Barking,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Barking,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
venues_encoded.shape

(2316, 234)

In [38]:
venues_encoded = venues_encoded.groupby('Neighborhood').mean().reset_index()
venues_encoded.head()

,Neighborhood,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus School,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Field,Hockey Rink,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soccer Stadium,Social Club,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Barking,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.018519,0.0,0.0,0.018519,0.000000,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.018519,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.037037,0.000000,0.0,0.0,0.018519,0.0,0.00,0.074074,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.00,0.018519,0.037037,0.0,0.000000,0.0,0.018519,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.055556,0.018519,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.074074,0.018519,0.000000,0.018519,0.0,0.018519,0.00,0.018519,0.0,0.0,0.0,0.074074,0.00

In [39]:
venues_encoded.shape

(24, 234)

In [40]:
X = venues_encoded.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=3, random_state=3)
kmeans.fit(X)

# check cluster labels generated for each row in the dataframe
set(kmeans.labels_)

{0, 1, 2}

In [49]:
# add Cluster Labels to Neighborhoods Data 
london_clusters = london_neighborhoods.drop('Station', axis=1)
london_clusters.insert(0, 'Cluster', kmeans.labels_)
london_clusters.head()

,Cluster,Latitude,Longitude,Neighborhood
0,0,51.538992,0.080424,Barking
1,2,51.511585,-0.103767,Blackfriars
2,0,51.400041,0.018277,Bromley South
3,1,51.511381,-0.090363,Cannon Street
4,2,51.507497,-0.123689,Charing Cross


#### Visualize the clustered neighborhoods using Folium

In [50]:
# create map of london
london_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_clusters['Latitude'], london_clusters['Longitude'], london_clusters['Neighborhood'], london_clusters['Cluster']):
    label = folium.Popup(str(poi) + ' :Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(london_clusters_map)
       
london_clusters_map

Now, continuing further we will find most common venues of in these neighborhoods and see what is common within each cluster

#### Get the top 10 most common venue categories in each neighborhood

In [51]:
cols = ['Neighborhood'] + ['Most Common Venue '+str(i) for i in range(1,11)]

neighborhood_top_venues = pd.DataFrame(columns=cols)

unique_neighborhoods = neighborhood_venues['Neighborhood'].unique()

for neighbor in unique_neighborhoods:
    df = neighborhood_venues[neighborhood_venues['Neighborhood'] == neighbor]
    top_venues = df['Venue Category'].value_counts().sort_values(ascending=False).head(10).index.to_list()
    
    if len(top_venues) >= 10:
        
        neighborhood_top_venues = neighborhood_top_venues.append({cols[0]:neighbor,
                                                              cols[1]:top_venues[0],
                                                              cols[2]:top_venues[1],
                                                              cols[3]:top_venues[2],
                                                              cols[4]:top_venues[3],
                                                              cols[5]:top_venues[4],
                                                              cols[6]:top_venues[5],
                                                              cols[7]:top_venues[6],
                                                              cols[8]:top_venues[7],
                                                              cols[9]:top_venues[8],
                                                              cols[10]:top_venues[9]}, ignore_index=True)
        
neighborhood_top_venues.head()

,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
0,Barking,Supermarket,Hotel,Coffee Shop,Grocery Store,Pub,Fast Food Restaurant,Discount Store,Pizza Place,Park,Chinese Restaurant
1,Blackfriars,Theater,Hotel,Coffee Shop,Performing Arts Venue,Restaurant,History Museum,Gym / Fitness Center,Art Gallery,Wine Bar,Park
2,Bromley South,Pub,Coffee Shop,Pizza Place,Clothing Store,Indian Restaurant,Park,Gym / Fitness Center,Supermarket,Café,Electronics Store
3,Cannon Street,Hotel,Coffee Shop,Cocktail Bar,Theater,Scenic Lookout,Grocery Store,Pizza Place,Gym / Fitness Center,Castle,Wine Bar
4,Charing Cross,Theater,Hotel,Lounge,Steakhouse,Bookstore,Plaza,Ice Cream Shop,Garden,Comic Shop,Pub


In [52]:
neighborhood_top_venues.shape

(24, 11)

#### Examine each cluster and determine how they are distinguished

In [53]:
london_cluster_venues = pd.merge(left=london_clusters, right=neighborhood_top_venues, how='left', on='Neighborhood')
london_cluster_venues.head()

,Cluster,Latitude,Longitude,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
0,0,51.538992,0.080424,Barking,Supermarket,Hotel,Coffee Shop,Grocery Store,Pub,Fast Food Restaurant,Discount Store,Pizza Place,Park,Chinese Restaurant
1,2,51.511585,-0.103767,Blackfriars,Theater,Hotel,Coffee Shop,Performing Arts Venue,Restaurant,History Museum,Gym / Fitness Center,Art Gallery,Wine Bar,Park
2,0,51.400041,0.018277,Bromley South,Pub,Coffee Shop,Pizza Place,Clothing Store,Indian Restaurant,Park,Gym / Fitness Center,Supermarket,Café,Electronics Store
3,1,51.511381,-0.090363,Cannon Street,Hotel,Coffee Shop,Cocktail Bar,Theater,Scenic Lookout,Grocery Store,Pizza Place,Gym / Fitness Center,Castle,Wine Bar
4,2,51.507497,-0.123689,Charing Cross,Theater,Hotel,Lounge,Steakhouse,Bookstore,Plaza,Ice Cream Shop,Garden,Comic Shop,Pub


In [55]:
# Cluster 0
london_cluster_venues[london_cluster_venues['Cluster'] == 0]

,Cluster,Latitude,Longitude,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
0,0,51.538992,0.080424,Barking,Supermarket,Hotel,Coffee Shop,Grocery Store,Pub,Fast Food Restaurant,Discount Store,Pizza Place,Park,Chinese Restaurant
2,0,51.400041,0.018277,Bromley South,Pub,Coffee Shop,Pizza Place,Clothing Store,Indian Restaurant,Park,Gym / Fitness Center,Supermarket,Café,Electronics Store
5,0,51.464459,-0.170518,Clapham Junction,Pub,Coffee Shop,Café,Italian Restaurant,Park,Bakery,Pizza Place,Cocktail Bar,Breakfast Spot,Grocery Store
6,0,51.375845,-0.092732,East Croydon,Pub,Coffee Shop,Grocery Store,Park,Clothing Store,Mediterranean Restaurant,Fast Food Restaurant,Café,Italian Restaurant,Indian Restaurant
14,0,51.461353,-0.303277,Richmond,Pub,Café,Garden,Park,Italian Restaurant,Coffee Shop,Scenic Lookout,Art Gallery,Restaurant,Hotel
15,0,51.531151,-0.077934,Shenfield,Coffee Shop,Pub,Bookstore,Café,Cocktail Bar,Italian Restaurant,Restaurant,Market,Bakery,Gym / Fitness Center
16,0,51.541289,-0.003547,Stratford,Café,Pub,Park,Art Gallery,Italian Restaurant,Grocery Store,Clothing Store,Bar,Cosmetics Shop,Restaurant
17,0,51.392411,-0.303999,Surbiton,Pub,Coffee Shop,Grocery Store,Café,Gym / Fitness Center,Park,Hotel,Italian Restaurant,Bar,Bakery
18,0,51.487483,-0.122930,Vauxhall,Café,Hotel,Park,Coffee Shop,Pub,Italian Restaurant,Restaurant,Supermarket,Cricket Ground,French Restaurant
22,0,51.476118,-0.166383,Watford Junction,Pub,Bakery,Coffee Shop,Café,French Restaurant,Italian Restaurant,Plaza,Park,Restaurant,Ice Cream Shop


In [56]:
# Cluster 1
london_cluster_venues[london_cluster_venues['Cluster'] == 1]

,Cluster,Latitude,Longitude,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
3,1,51.511381,-0.090363,Cannon Street,Hotel,Coffee Shop,Cocktail Bar,Theater,Scenic Lookout,Grocery Store,Pizza Place,Gym / Fitness Center,Castle,Wine Bar
7,1,51.528287,-0.133875,Euston,Hotel,Coffee Shop,Bookstore,Theater,Cocktail Bar,Bakery,Beer Bar,Tapas Restaurant,Zoo Exhibit,Wine Bar
8,1,51.512054,-0.080184,Fenchurch Street,Coffee Shop,Hotel,Cocktail Bar,Scenic Lookout,Italian Restaurant,Restaurant,Steakhouse,Pizza Place,Pub,French Restaurant
9,1,51.532395,-0.123022,King's Cross,Coffee Shop,Hotel,Bookstore,Café,Breakfast Spot,Middle Eastern Restaurant,Bakery,Canal,Pub,Cocktail Bar
10,1,51.517134,-0.081643,Liverpool Street,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Steakhouse,Food Truck,Pizza Place,Beer Bar,French Restaurant,Bagel Shop
11,1,51.508049,-0.087672,London Bridge,Coffee Shop,Hotel,Cocktail Bar,Scenic Lookout,Italian Restaurant,Grocery Store,Steakhouse,Seafood Restaurant,Restaurant,Café
12,1,51.522059,-0.150455,Marylebone,Hotel,Coffee Shop,Juice Bar,Art Gallery,Clothing Store,Modern European Restaurant,Indian Restaurant,Hotel Bar,Cocktail Bar,Bakery
13,1,51.517086,-0.177542,Paddington,Hotel,Café,Pub,Garden,Gym / Fitness Center,Restaurant,Coffee Shop,Persian Restaurant,Juice Bar,Gastropub


In [57]:
# Cluster 2
london_cluster_venues[london_cluster_venues['Cluster'] == 2]

,Cluster,Latitude,Longitude,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
1,2,51.511585,-0.103767,Blackfriars,Theater,Hotel,Coffee Shop,Performing Arts Venue,Restaurant,History Museum,Gym / Fitness Center,Art Gallery,Wine Bar,Park
4,2,51.507497,-0.123689,Charing Cross,Theater,Hotel,Lounge,Steakhouse,Bookstore,Plaza,Ice Cream Shop,Garden,Comic Shop,Pub
19,2,51.496369,-0.143152,Victoria,Hotel,Japanese Restaurant,Lounge,Café,Indian Restaurant,Plaza,Department Store,Boutique,French Restaurant,Park
20,2,51.503002,-0.113981,Waterloo,Theater,Hotel,Plaza,Art Gallery,Garden,Art Museum,Scenic Lookout,Sushi Restaurant,Bakery,Park
21,2,51.504217,-0.108203,Waterloo East,Theater,Hotel,Art Gallery,Italian Restaurant,Coffee Shop,Scenic Lookout,Garden,History Museum,Grocery Store,Sushi Restaurant


Cluster 0: Outer Neighborhoods (furthest from center) with pubs and cafes. Also, Italian Restaurants seem to quite popular.  
  
Cluster 1: Inner Neighborhoods (closer to center) with Hotels, Coffee Shops, Cocktail Bars plus a variety of Restaurants. This cluster seems to offer a lot of variety and lot of things to do attracting people from all walks of life.  
  
Cluster 2: Innermost Neighborhoods (closest to center) with Theaters, Hotels and Art Gallries and Museums. This cluster seems to be the cultural and artistic center of London.

***Cluster 1 seems to be a good choice to open a restaurant or cloud kitchen***

## Recommend Locations to open Cloud Kitchen

#### Find top 5 neighborhoods with Indian Restaurants

In [64]:
neighborhood_venues_indian = neighborhood_venues[neighborhood_venues['Venue Category'].str.contains('Indian')]
neighborhood_venues_indian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
185,Bromley South,51.400041,0.018277,Cinnamon Culture,51.414196,0.020883,Indian Restaurant
200,Bromley South,51.400041,0.018277,Tamasha,51.406949,0.027044,Indian Restaurant
210,Bromley South,51.400041,0.018277,Shampan Bromley,51.392338,0.035468,Indian Restaurant
303,Cannon Street,51.511381,-0.090363,Gunpowder,51.518436,-0.074732,Indian Restaurant
345,Charing Cross,51.507497,-0.123689,Dishoom,51.512383,-0.126949,Indian Restaurant


In [59]:
neighborhood_venues_indian['Neighborhood'].value_counts().head(5)

East Croydon     5
Victoria         4
Bromley South    3
Marylebone       3
Waterloo         2
Name: Neighborhood, dtype: int64

#### Find the neighborhoods with Indian Restaurants amongst Top 10 venues

In [65]:
# Get rows from top venues dataframe where 'Indian Restaurant' is one of the venues
neighborhood_top_venues_indian = london_cluster_venues[london_cluster_venues.isin(['Indian Restaurant'])]

In [66]:
# Get index of rows as found above
neighborhood_top_venues_indian[neighborhood_top_venues_indian.notnull().any(axis=1)].index

Int64Index([2, 6, 12, 19], dtype='int64')

In [69]:
# Filter rows with above found index
london_cluster_venues.iloc[[2, 6, 12, 19]]

,Cluster,Latitude,Longitude,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
2,0,51.400041,0.018277,Bromley South,Pub,Coffee Shop,Pizza Place,Clothing Store,Indian Restaurant,Park,Gym / Fitness Center,Supermarket,Café,Electronics Store
6,0,51.375845,-0.092732,East Croydon,Pub,Coffee Shop,Grocery Store,Park,Clothing Store,Mediterranean Restaurant,Fast Food Restaurant,Café,Italian Restaurant,Indian Restaurant
12,1,51.522059,-0.150455,Marylebone,Hotel,Coffee Shop,Juice Bar,Art Gallery,Clothing Store,Modern European Restaurant,Indian Restaurant,Hotel Bar,Cocktail Bar,Bakery
19,2,51.496369,-0.143152,Victoria,Hotel,Japanese Restaurant,Lounge,Café,Indian Restaurant,Plaza,Department Store,Boutique,French Restaurant,Park


#### Observations and Conclusion

- Based on cluster analysis, we found Cluster 1 to be most suitable  
  

- Top 5 neighborhoods with Indian Restaurants are East Croydon, Victoria, Bromley South, Charing Cross, Marylebone. 
  
  
- We also found that among above East Croydon, Victoria, Bromley South, Marylebone also have Indian Restaurant amongst their top 10 venues. Out of these Marylebone falls under Cluster 1.  
  
  
- Having Indian Restaurants featuring in Top 10 venues, would also mean that these areas would have a good amount of Indian or Indian origin population, which fits our target audience as well.   
  
  
- Looking at the map, East Croydon and Bromley South are far down south, so we would exclude these.

Based on these observations, we would recommend one of **Marylebone or Victoria** as the location for opening our new **'Goila Butter Chicken**' Cloud Kitchen.
  
Now, if we check the actual location of 'Goila Butter Chicken' cloud kitchen in London [here](https://goilabutterchicken.co.uk/), we see that it has been opened in **Marylebone**
  
Our analysis of London Neighborhoods has lead to satisfactory results, so now we will use similar analysis to find suitable locations in Melbourne and New York as well